# RAG Chatbot - Step 2: Document Chunking & Embedding

## Objectives:
- Split documents into smaller chunks
- Generate embeddings for each chunk
- Prepare data for vector database

## From EDA:
- 20 documents, average 5,797 words
- Recommended chunk size: 150 words
- Recommended overlap: 30 words

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import json
from typing import List, Dict
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print('✅ Libraries imported successfully!')

✅ Libraries imported successfully!


## 1. Load Data & EDA Summary

In [3]:
# Load documents
documents_df = pd.read_csv('../data_rag/documents.csv')

# Load EDA summary
with open('eda_summary.json', 'r') as f:
    eda_summary = json.load(f)

print('✅ Data loaded!')
print(f'📄 Number of documents: {len(documents_df)}')
print(f'\n📊 EDA Summary:')
for key, value in eda_summary.items():
    print(f'  - {key}: {value}')

✅ Data loaded!
📄 Number of documents: 20

📊 EDA Summary:
  - num_documents: 20
  - avg_doc_length: 5797.1
  - median_doc_length: 3137.5
  - num_single_passage_q: 40
  - num_multi_passage_q: 40
  - num_no_answer_q: 40
  - recommended_chunk_size: 300
  - recommended_overlap: 50


In [4]:
# Check column names
print('📝 Available columns:')
print(documents_df.columns.tolist())

# Set text column name - adjust if needed
text_column = 'text'  # ⚠️ Change this to match your column name
id_column = 'id'      # ⚠️ Change this if you have a document ID column

# If no ID column exists, create one
if id_column not in documents_df.columns:
    documents_df['id'] = documents_df.index
    id_column = 'id'
    print('\n⚠️ No ID column found, created index-based IDs')

print(f'\n✅ Using columns:')
print(f'  - Text: {text_column}')
print(f'  - ID: {id_column}')

📝 Available columns:
['index', 'source_url', 'text']

⚠️ No ID column found, created index-based IDs

✅ Using columns:
  - Text: text
  - ID: id


## 2. Document Chunking
###  กฎทั่วไป:

chunk_size = median × 5-10%   ดังนั้น  chunk_size = 3,137 × 0.05 = 157 words

overlap = chunk_size × 15-20%  ดังนั้น chunk_size = 150 → overlap = 30 words (20%)
### Strategy:
- **Chunk size**: 150 words (from EDA)
- **Overlap**: 30 words (to preserve context)
- **Method**: Word-based splitting (simple and effective)

In [13]:
def calculate_chunk_size(documents_df, text_column='text'):
    # คำนวณ median (ดีกว่า average)
    word_counts = documents_df[text_column].str.split().str.len()
    median = word_counts.median()
    
    # Chunk = 5-7.5% ของ median
    chunk_size = int(median * 0.05)  # หรือ 0.075
    chunk_overlap = int(chunk_size * 0.20)
    
    return chunk_size, chunk_overlap
chunk_size, chunk_overlap = calculate_chunk_size(documents_df, text_column='text')
print(f'\n🔍 Calculated chunk size: {chunk_size} words')
print(f'🔍 Calculated overlap: {chunk_overlap} words')


🔍 Calculated chunk size: 156 words
🔍 Calculated overlap: 31 words


In [14]:
def chunk_text(text: str, chunk_size:int , overlap: int) -> List[str]:
    """
    Split text into overlapping chunks based on word count.
    
    Args:
        text: Input text to chunk
        chunk_size: Number of words per chunk
        overlap: Number of overlapping words between chunks
    
    Returns:
        List of text chunks
    """
    # Split text into words
    words = text.split()
    
    # If text is shorter than chunk size, return as single chunk
    if len(words) <= chunk_size:
        return [text]
    
    chunks = []
    start = 0
    
    while start < len(words):
        # Get chunk
        end = start + chunk_size
        chunk_words = words[start:end]
        
        # Join words back to text
        chunk = ' '.join(chunk_words)
        chunks.append(chunk)
        
        # Move start position (with overlap)
        start += chunk_size - overlap
        
        # Break if we've reached the end
        if end >= len(words):
            break
    
    return chunks

# Test the function
test_text = "This is a test sentence. " * 100
test_chunks = chunk_text(test_text, chunk_size= chunk_size, overlap=chunk_overlap)
print(f'✅ Chunking function ready!')
print(f'Test: {len(test_text.split())} words → {len(test_chunks)} chunks')

✅ Chunking function ready!
Test: 500 words → 4 chunks


In [15]:
print(f'🔪 Chunking documents...')
print(f'  - Chunk size: {chunk_size} words')
print(f'  - Overlap: {chunk_overlap} words')
print()

all_chunks = []
chunk_metadata = []

for idx, row in tqdm(documents_df.iterrows(), total=len(documents_df), desc='Chunking'):
    doc_id = row[id_column]
    text = row[text_column]
    
    # Create chunks
    chunks = chunk_text(text, chunk_size=chunk_size, overlap=chunk_overlap)
    
    # Store chunks with metadata
    for chunk_idx, chunk in enumerate(chunks):
        all_chunks.append(chunk)
        chunk_metadata.append({
            'doc_id': doc_id,
            'chunk_id': chunk_idx,
            'chunk_text': chunk,
            'chunk_length': len(chunk.split())
        })

print(f'\n✅ Chunking complete!')
print(f'📄 Total documents: {len(documents_df)}')
print(f'📦 Total chunks: {len(all_chunks)}')
print(f'📊 Average chunks per document: {len(all_chunks) / len(documents_df):.1f}')

🔪 Chunking documents...
  - Chunk size: 156 words
  - Overlap: 31 words



Chunking: 100%|██████████| 20/20 [00:00<00:00, 1111.29it/s]


✅ Chunking complete!
📄 Total documents: 20
📦 Total chunks: 933
📊 Average chunks per document: 46.6


In [16]:
# Analyze chunk statistics
chunk_df = pd.DataFrame(chunk_metadata)

print('📊 Chunk Statistics:')
print(chunk_df['chunk_length'].describe())

# Show sample chunks
print('\n📝 Sample Chunks:')
print('='*80)
for i in range(min(3, len(all_chunks))):
    print(f'\nChunk {i} (Doc {chunk_df.iloc[i]["doc_id"]}, Chunk {chunk_df.iloc[i]["chunk_id"]}):')
    print(f'Length: {chunk_df.iloc[i]["chunk_length"]} words')
    print(all_chunks[i][:200] + '...')
    print('-'*80)

📊 Chunk Statistics:
count    933.000000
mean     154.603430
std       10.805998
min       35.000000
25%      156.000000
50%      156.000000
75%      156.000000
max      156.000000
Name: chunk_length, dtype: float64

📝 Sample Chunks:

Chunk 0 (Doc 0, Chunk 0):
Length: 156 words
Bullet Kin Bullet Kin are one of the most common enemies. They slowly walk towards the player, occasionally firing a single bullet. They can flip tables and use them as cover. They will also deal cont...
--------------------------------------------------------------------------------

Chunk 1 (Doc 0, Chunk 1):
Length: 156 words
if there are any unoccupied Minecarts within the room, they will take priority by walking towards them to ride in. Trivia Bullet Kin wield Magnums. Assault-rifle wielding Bullet Kin wield AK-47s. Inca...
--------------------------------------------------------------------------------

Chunk 2 (Doc 0, Chunk 2):
Length: 156 words
game Riverbond. Bullet Kin also has a cameo as lower and upper

## 3. Generate Embeddings

### Embedding Model:
- Using **sentence-transformers** (all-MiniLM-L6-v2)
- Dimension: 384
- Fast and efficient for retrieval tasks

In [17]:
# Import sentence transformers
from sentence_transformers import SentenceTransformer

# Load embedding model
print('⏳ Loading embedding model...')
model_name = 'BAAI/bge-base-en-v1.5'
embedding_model = SentenceTransformer(model_name)

print(f'✅ Model loaded: {model_name}')
print(f'📐 Embedding dimension: {embedding_model.get_sentence_embedding_dimension()}')

⏳ Loading embedding model...
✅ Model loaded: BAAI/bge-base-en-v1.5
📐 Embedding dimension: 768


In [18]:
# Test embedding
test_text = "This is a test sentence for embedding."
test_embedding = embedding_model.encode(test_text)

print('🧪 Test Embedding:')
print(f'  - Input: "{test_text}"')
print(f'  - Output shape: {test_embedding.shape}')
print(f'  - First 5 values: {test_embedding[:5]}')

🧪 Test Embedding:
  - Input: "This is a test sentence for embedding."
  - Output shape: (768,)
  - First 5 values: [ 0.01576559 -0.03737001  0.02165025  0.0359973   0.07341885]


In [19]:
# Generate embeddings for all chunks
print(f'🔄 Generating embeddings for {len(all_chunks)} chunks...')
print('⚠️ This may take a few minutes depending on your hardware.')
print()

# Encode in batches for efficiency
batch_size = 32
embeddings = embedding_model.encode(
    all_chunks,
    batch_size=batch_size,
    show_progress_bar=True,
    convert_to_numpy=True
)

print(f'\n✅ Embeddings generated!')
print(f'📊 Shape: {embeddings.shape}')
print(f'💾 Memory: {embeddings.nbytes / (1024**2):.2f} MB')

🔄 Generating embeddings for 933 chunks...
⚠️ This may take a few minutes depending on your hardware.



Batches:   0%|          | 0/30 [00:00<?, ?it/s]


✅ Embeddings generated!
📊 Shape: (933, 768)
💾 Memory: 2.73 MB


## 4. Save Chunks and Embeddings

In [20]:
# Create output directory
import os
os.makedirs('models/chunks', exist_ok=True)

# Save embeddings
np.save('models/chunks/embeddings.npy', embeddings)
print('✅ Embeddings saved to: models/chunks/embeddings.npy')

# Save chunk metadata
chunk_df.to_csv('models/chunks/chunk_metadata.csv', index=False)
print('✅ Metadata saved to: models/chunks/chunk_metadata.csv')

# Save chunks as JSON (for easier access)
chunks_data = {
    'chunks': all_chunks,
    'metadata': chunk_metadata,
    'config': {
        'chunk_size': chunk_size,
        'overlap': chunk_overlap,
        'embedding_model': model_name,
        'embedding_dim': embedding_model.get_sentence_embedding_dimension(),
        'num_chunks': len(all_chunks),
        'num_documents': len(documents_df)
    }
}

with open('models/chunks/chunks_data.json', 'w', encoding='utf-8') as f:
    json.dump(chunks_data, f, ensure_ascii=False, indent=2)
print('✅ Chunks data saved to: models/chunks/chunks_data.json')

✅ Embeddings saved to: models/chunks/embeddings.npy
✅ Metadata saved to: models/chunks/chunk_metadata.csv
✅ Chunks data saved to: models/chunks/chunks_data.json


## 5. Quality Check

In [21]:
# Verify embeddings quality
from sklearn.metrics.pairwise import cosine_similarity

# Test: Similar chunks should have high similarity
# Take first 5 chunks from same document
sample_chunks = all_chunks[:5]
sample_embeddings = embeddings[:5]

# Calculate pairwise similarities
similarities = cosine_similarity(sample_embeddings)

print('🔍 Quality Check: Pairwise Similarities (first 5 chunks)')
print('Expected: Chunks from same document should have higher similarity')
print()
print('Similarity Matrix:')
print(pd.DataFrame(
    similarities,
    columns=[f'Chunk {i}' for i in range(5)],
    index=[f'Chunk {i}' for i in range(5)]
).round(3))

🔍 Quality Check: Pairwise Similarities (first 5 chunks)
Expected: Chunks from same document should have higher similarity

Similarity Matrix:
         Chunk 0  Chunk 1  Chunk 2  Chunk 3  Chunk 4
Chunk 0    1.000    0.829    0.756    0.733    0.672
Chunk 1    0.829    1.000    0.791    0.749    0.685
Chunk 2    0.756    0.791    1.000    0.824    0.667
Chunk 3    0.733    0.749    0.824    1.000    0.714
Chunk 4    0.672    0.685    0.667    0.714    1.000


In [22]:
# Test: Find most similar chunk to a query
test_query = "What is machine learning?"
query_embedding = embedding_model.encode(test_query)

# Calculate similarities
query_similarities = cosine_similarity([query_embedding], embeddings)[0]

# Get top 3 most similar chunks
top_k = 3
top_indices = np.argsort(query_similarities)[::-1][:top_k]

print(f'\n🔎 Test Retrieval: "{test_query}"')
print(f'\nTop {top_k} most similar chunks:')
print('='*80)

for rank, idx in enumerate(top_indices, 1):
    score = query_similarities[idx]
    chunk_info = chunk_metadata[idx]
    chunk_text = all_chunks[idx]
    
    print(f'\n#{rank} | Similarity: {score:.4f}')
    print(f'Doc ID: {chunk_info["doc_id"]} | Chunk: {chunk_info["chunk_id"]}')
    print(f'Text: {chunk_text[:150]}...')
    print('-'*80)


🔎 Test Retrieval: "What is machine learning?"

Top 3 most similar chunks:

#1 | Similarity: 0.6003
Doc ID: 2 | Chunk: 19
Text: could have written a framework for loading, unloading, and executing the models, passing context and queries to the models, and integrating the vector...
--------------------------------------------------------------------------------

#2 | Similarity: 0.5860
Doc ID: 13 | Chunk: 7
Text: for i in range(0, len(train_data) - ctx_len, ctx_len)]) X_val = mx.array([val_data[i:i+ctx_len] for i in range(0, len(val_data) - ctx_len, ctx_len)]) ...
--------------------------------------------------------------------------------

#3 | Similarity: 0.5809
Doc ID: 6 | Chunk: 13
Text: for a variety of tasks. Individual models may be integrated into a large number of AI systems. It is important that a provider wishing to build upon a...
--------------------------------------------------------------------------------


## 6. Summary

In [24]:
# Create summary
chunking_summary = {
    'num_documents': len(documents_df),
    'num_chunks': len(all_chunks),
    'chunks_per_doc': len(all_chunks) / len(documents_df),
    'chunk_size': chunk_size,
    'overlap': chunk_overlap,
    'embedding_model': model_name,
    'embedding_dim': int(embedding_model.get_sentence_embedding_dimension()),
    'avg_chunk_length': float(chunk_df['chunk_length'].mean()),
    'output_files': {
        'embeddings': 'models/chunks_bge/embeddings.npy',
        'metadata': 'models/chunks_bge/chunk_metadata.csv',
        'chunks_data': 'models/chunks_bge/chunks_data.json'
    }
}

# Save summary
with open('models/chunks_bge/chunking_summary.json', 'w') as f:
    json.dump(chunking_summary, f, indent=2)

print('\n' + '='*80)
print('📊 CHUNKING & EMBEDDING SUMMARY')
print('='*80)
for key, value in chunking_summary.items():
    if key != 'output_files':
        print(f'{key}: {value}')

print('\n📁 Output Files:')
for key, path in chunking_summary['output_files'].items():
    print(f'  - {key}: {path}')

print('\n✅ Step 2 Complete!')
print('💾 Summary saved to: models/chunks/chunking_summary.json')


📊 CHUNKING & EMBEDDING SUMMARY
num_documents: 20
num_chunks: 933
chunks_per_doc: 46.65
chunk_size: 156
overlap: 31
embedding_model: BAAI/bge-base-en-v1.5
embedding_dim: 768
avg_chunk_length: 154.60342979635584

📁 Output Files:
  - embeddings: models/chunks_bge/embeddings.npy
  - metadata: models/chunks_bge/chunk_metadata.csv
  - chunks_data: models/chunks_bge/chunks_data.json

✅ Step 2 Complete!
💾 Summary saved to: models/chunks/chunking_summary.json
